In [3]:
import numpy as np
import random
import sys
import time
import shutil
from importlib import import_module
from numbers import Number

import torch
from torch.utils.data import Sampler, DataLoader


from utils import Logger, load_pretrain
from lanegcn import get_model
import matplotlib.pyplot as plt
import torch.nn as nn

In [2]:
config, Dataset, collate_fn, net, loss, post_process, opt = get_model()
def worker_init_fn(pid):
    np_seed = int(pid)
    np.random.seed(np_seed)
    random_seed = np.random.randint(2 ** 32 - 1)
    random.seed(random_seed)

dataset = Dataset('./dataset/preprocess/train_crs_dist6_angle90.p', config, train=True)
train_loader = DataLoader(
        dataset,
        batch_size=config["batch_size"],
        num_workers=config["workers"],
        shuffle=False,   # True: At each epoch, reorder the data
        collate_fn=collate_fn,
        pin_memory=True,
        worker_init_fn=worker_init_fn,   # The next 36 were thrown away
        drop_last=True,
    )

In [4]:
data = {}
for i, data in enumerate(train_loader):
    data = dict(data)
    break


print(data.keys())      

dict_keys(['city', 'orig', 'gt_preds', 'has_preds', 'theta', 'rot', 'feats', 'ctrs', 'graph', 'trajs2', 'traj1'])


In [5]:
from lanegcn import actor_gather
actors, actor_idcs = actor_gather(data["feats"])
print(len(actors),actors.size())

414 torch.Size([414, 3, 20])


In [7]:
encoder = nn.LSTM(3, 32, 1)
output, state = encoder(actors.view(-1, len(actors), 3))

In [11]:
print(output.size())
print(len(state))
print(state[0].size())

torch.Size([20, 414, 32])
2
torch.Size([1, 414, 32])
